In [ ]:
# import settings and functions
%run ./../../imports.ipynb

# resolution
res_multiply = 1
params.p_resolution = [1000*res_multiply,1100*res_multiply]
params.crop_top_x_percent_pdf = 10

In [3]:
rand = 0

rand_name = ""
if rand:
    rand_name = "rand_"
else:
    rand_name = "zero_"

## What mesh?

Copy your choice to the next cell

for SquareTop:
```
analytical_solution_tag = "-ana_square_top"
generate_config = generateConfig_squareTop
generate_mesh = generateMesh_squareTop
```

for SquareSinCos:
```
analytical_solution_tag = "-ana_square_sincos"
generate_config = generateConfig_squareSinCos
generate_mesh = generateMesh_squareSinCos
```

In [4]:
# # Change according to instruction above
# analytical_solution_tag = "-ana_mexi_hat"
# generate_config = generateConfig_squareMexiHat
# generate_mesh = generateMesh_squareMexiHat

# analytical_solution_tag = "-ana_square_top"
# generate_config = generateConfig_squareTop
# generate_mesh = generateMesh_squareTop

# analytical_solution_tag = "-ana_square_sincos"
# generate_config = generateConfig_squareSinCos
# generate_mesh = generateMesh_squareSinCos

analytical_solution_tag = "-ana_L_shape"
generate_config = generateConfig_Lshape
generate_mesh = generateMesh_Lshape

## Analysis setup

In [5]:
prefix = "c6_L_0_"

# which executable?

exe = hdiv_data_driven_diffusion_snes
sumanalys = "sumanalys.csv"
ana_name = "ana_square_mexi_mixed_order"

# ana_compare_exe = [data_driven_diffusion_snes, classic_diffusion]
# ana_compare_name = ["ana_square_mexi_dd", "ana_square_mexi_classic"]
# # ana_compare_name = ["ana_square_mexi_mixed"]
# ana_compare_sum = ["sumanalys.csv", "sumanalys.csv", "FEM_errors.csv", "sumanalys.csv"]

# Convergence analysis parameters
order_list = [1, 2, 3] # approximation order p
elem_size_list = [0.08, 0.04, 0.02] # element size h
# elem_size_list = [0.1, 0.05, 0.025] # element size h
# order_list = [2, 3] # approximation order p
# elem_size_list = [1./3., 1./7., 1./13.] # element size h
params.triangle_mesh = True
params.nproc = 1 # number of processors
jumps = ""
if params.nproc == 1:
    jumps = "-get_jumps"
# jumps = "-get_jumps"

run_test = True
run_analysis = True

# params.triangle_mesh = False

run_test = False
run_analysis = False

# naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
#           "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter", "errorIndicatorGrad", "errorIndicatorDiv", "jumpL2", "jumpHdiv", "eleNum"]
naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorEstimator",
          "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter"]

error_name_list = ["L2norm", "H1seminorm", "fluxErr"]
error_label_list = [(r'Global error $L^2$-norm'),
               (r'Global error $H^1$-seminorm'), (r'Global Flux error')]

In [6]:
params.conductivity = 1.0 # linear conductivity
# params.element_size = elem_size_list[0] # element size in the regular mesh
params.element_size = 0.1
params.order = 2 # approximation order for displacements

# params.triangle_mesh = False # use triangular mesh

# Pre-processing parameters
# params.mesh_file = "square_mexi"
params.mesh_file = "L_shape"
params.length_x = 1
params.length_y = 1
params.length_z = 0
params.show_mesh = True


# solution parameters
params.log_file = "log" # log file name 


## Run test

In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

In [ ]:
# Testing mesh generation
if run_test:
    params.show_mesh = True
    generate_config(params)
    generate_mesh(params)

# stop the display
display.stop()


In [ ]:
display.stop()

In [ ]:
params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
!{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1

In [ ]:
# recreate dataset
recreate_dataset = False
csv_name = "dummy_tree"

dummy_noise = 0.02

dummy_count = 1e4

if recreate_dataset:
    !{create_csv_dataset} -output_file {csv_name}.csv -my_dummy_noise_q 0 -my_dummy_noise_k {dummy_noise} -my_dummy_k 1 -my_dummy_range_dp 9.0 -my_dummy_count {int(dummy_count)}

    scal = np.genfromtxt('./scaling.in')
    print(scal)

data = pd.read_csv(csv_name+".csv", index_col=False)
headers = []
headers = data.columns.values
print(headers)

In [12]:
# # plot the dataset
# plt.figure()
# # size of markers is 2
# plt.plot(data[headers[0]], data[headers[2]], 'ro', markersize=2)
# plt.xlabel(headers[0])
# plt.ylabel(headers[2])
# plt.show()

In [13]:
# noise_list = [0.0, 0.01, 0.02, 0.05, 0.1]
# noise_list = [0.02, 0.05, 0.1]
# ana_name = "noise_sincos_"

# ana_name_list = []

# for noise in noise_list:
#     ana_name_list.append(ana_name + str(noise))
    
# dummy_count_list = [1e3, 1e4, 1e5, 1e6, 1e7]
# dummy_count_list = [1e3, 1e4, 1e5, 1e6]
dummy_count = 1e6

# realisations = 30
monte_num = 0

monte_control = f"-monte_carlo {monte_num} -monte_patch_number 2"


refinement_style = 0
ref_iter_num = 0
refinement_control = ""
ref = False

if ref:
    print("refinement: ")

    # refinement style: 
    # 0 - no refinement
    # 1 - p refinement (order)
    # 2 - h refinement (mesh)
    # 3 - hp refinement
    refinement_style = 3

    # number of refinements
    ref_iter_num = 1

    # refine mesh due to error indicators only on boundary
    boundary_ony = "-refine_h_boundary_only"

    # order refinement control
    # > ord_ref_control * average error -> refine
    ord_ref_control = 3.0
    # mesh refinement control
    # > mesh_ref_control * average error -> refine
    mesh_ref_control = 4.0

    # # DD refinement control
    # # d > tol_distance_ref * d_ave -> do not refine
    # tol_distance_ref = 1.0
    # # d > tol_distance_var_ref * d_var 
    # # &&
    # # d_var > tol_distance_var_ave * d_ave
    # #  -> refine
    # tol_distance_var_ref = 2.0
    # tol_distance_var_ave = 0.5

    refinement_control = f"-refinement_style {refinement_style} -ref_iter_num {ref_iter_num} -ref_control {ord_ref_control} -ref_control_mesh {mesh_ref_control} {boundary_ony}"
    # -tol_distance_ref {tol_distance_ref} -tol_distance_var_ref {tol_distance_var_ref} -tol_distance_var_ave {tol_distance_var_ave} 

In [14]:
if run_analysis:
    !rm sumanalys.csv # making sure the previous analysis doesn't affect this one

    # loop over noise_lisgt and ana_name_list
    # for (noise_in, ana_csv_name) in zip(noise_list, ana_name_list):
    !rm out*
        # !{create_csv_dataset} -output_file {csv_name}.csv -my_dummy_noise_q 0 -my_dummy_noise_k {noise_in} -my_dummy_k 1 -my_dummy_range_dp 9.0 -my_dummy_count {int(dummy_count)}

        # for i in range(realisations):
    !{exe} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag} -csv_tree_file {csv_name}.csv -write_long_error_file -rand_ini {rand} {monte_control} {refinement_control}

        # !mv ./sumanalys.csv ./{ana_csv_name}.csv
        # !mbconvert out_sigma_ele_29.h5m {noise_in*100}_out_sigma_29.vtk
        # !mbconvert out_monte_ele_errors_29.h5m {noise_in*100}_out_monte_ele_errors_29.vtk
        # !mbconvert out_before_perturb_29.h5m {noise_in*100}_out_before_perturb_29.vtk
        # !mbconvert out_after_perturb_29.h5m {noise_in*100}_out_after_perturb_29.vtk
        # !mv ./sumanalys_monte_sigma.csv ./{ana_csv_name}_sigma.csv


In [ ]:
!cat ./sumanalys.csv

### Plot results

In [16]:
# %run ./../L_plot_save_sigmas.ipynb

In [ ]:
# print scatter for dataset from dummy_tree.csv

# read the data from dummy_tree.csv file with naming defined by headers
data = pd.read_csv("dummy_tree.csv", names=headers, index_col=False, skiprows=1)
print(data)

# plot row 0 vs row 2
plt.figure(figsize=(4.5,4.5))
# size of the points is 1
plt.scatter(data[headers[0]], data[headers[2]], s=1)
plt.xlabel(label_gradient_g_x)
plt.ylabel(label_flux_x)
plt.grid(True, ls=':')
plt.tight_layout()
plt.savefig("c6_uniques_tree.png", dpi=300)
plt.show()

# plot row 1 vs row 3
plt.figure()
plt.scatter(data[headers[1]], data[headers[3]], s=1)
plt.xlabel(headers[1])
plt.ylabel(headers[3])
plt.grid(True, ls=':')
plt.show()


In [ ]:
# mbconvert out_iteration_ h5m file with the biggest number to vtk file
# find last created out_iteration_ h5m file
files = !ls -tr out_iteration_*
last_file = files[-1]
print(last_file)

if run_analysis:
    # convert file 
    !mbconvert {last_file} {rand_name}last_iteration.vtk
    !mbconvert out_error_1000.h5m {rand_name}error_1000.vtk

In [ ]:
print("Done")

In [ ]:
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

params.show_mesh = True
params.show_field = "G"
params.field_part = -1
params.p_cmap = color_gradient
vtk_file = rand_name+"last_iteration"
print(vtk_file)
params.show_file = vtk_file
params.p_save = "c6_unique_" + rand_name + "G.pdf"
show_results(params)


params.field_part = -1

display.stop()

In [ ]:
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

params.show_mesh = True
params.show_field = "T"
params.p_cmap = color_temperature
# params.clim = [0.0, 1.27]
vtk_file = rand_name+"last_iteration"
print(vtk_file)
print("before perturbation")
params.show_file = vtk_file
params.p_save = "c6_unique_" + rand_name + "T.pdf"
show_results(params)

params.field_part = -1
params.clim = None

display.stop()

In [ ]:
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

params.show_mesh = True
params.show_field = "Q"
params.field_part = -1
params.p_cmap = color_flux
vtk_file = rand_name+"last_iteration"
print(vtk_file)
print("before perturbation")
params.show_file = vtk_file
params.p_save = "c6_unique_" + rand_name + "Q.pdf"
show_results(params)


params.field_part = -1

display.stop()

In [ ]:
# plot for the error estimators for {noise_in*100}_out_monte_ele_errors_29.vtk

display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

params.show_field = "ERROR_ESTIMATOR"
# params.show_field = "DD_DISTANCE_VAR"
# params.show_field = "ORDER"
params.show_edges = False
params.p_cmap = 'jet'

params.clim = [0.0, 0.0186]

vtk_file = rand_name+"error_1000"
print(vtk_file)
print("error estimator")
params.show_file = vtk_file
params.p_save = "c6_unique_" + rand_name + "err_est.pdf"
show_results(params)


params.clim = None

display.stop()